In [1]:
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *

from GeospatialFM.utils import *
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from tqdm import trange, tqdm
from matplotlib import pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
exp_name = 'debug'
args = ["--exp_name", exp_name, "--config_file", "GeospatialFM/configs/mae_cm.yaml", "--device_ids", "1", "7", "--debug"]
args = get_args_parser().parse_args(args)

In [3]:
if args.debug:
    logging.basicConfig(level=logging.INFO)
cfg, _ = setup(args)

In [4]:
cfg.LOSS['MAE']

{'scale': 1.0, 'recon_all': True, 'cross_modal_recon': False, 'channel_reweight': False}

In [5]:
training_args = dict(
    device_ids = args.device_ids,
    device = 'cpu',
    precision = None,
    accum_freq = cfg['TRAINER']['gradient_accumulation_steps'],
    grad_clip_norm = None,
    log_every_n_steps = cfg['TRAINER']['logging_steps'],
    wandb = cfg['TRAINER']['report_to'] == 'wandb',
    batch_size = cfg['TRAINER']['per_device_train_batch_size'],
    val_frequency = 1,
    epochs = cfg['TRAINER']['num_train_epochs'],
    save_logs = True,
    checkpoint_path = cfg['TRAINER']['logging_dir'],
    mask_ratio = cfg['MODEL']['mask_ratio']
)

In [6]:
training_args = argparse.Namespace(**training_args)
training_args.device = f'cuda:{training_args.device_ids[0]}'

In [17]:
model = construct_mae(cfg.MODEL)
model = model.to(training_args.device)
if len(training_args.device) > 1:
    model = torch.nn.DataParallel(model, device_ids=training_args.device_ids)

In [25]:
data = get_data(cfg)

### Train

In [9]:
# steps = data['train'].dataloader.num_batches * cfg['TRAINER']['num_train_epochs']
# optimizer = torch.optim.AdamW(model.parameters(), lr=cfg['TRAINER']['learning_rate'], weight_decay=cfg['TRAINER']['weight_decay'])
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=steps, eta_min=1e-7)
# loss = get_loss_list(cfg.LOSS)

In [10]:
# train one iter
# device = torch.device(training_args.device)
# autocast = get_autocast(training_args.precision)
# model.train()

# dataloader = data['train'].dataloader
# num_batches_per_epoch = dataloader.num_batches // training_args.accum_freq
# sample_digits = math.ceil(math.log(dataloader.num_samples + 1, 10))

# batch = next(iter(dataloader))

In [11]:
# images, radar, label = batch['image'], batch['radar'], batch['label']
# images = images.to(device=device, non_blocking=True)
# radar = radar.to(device=device, non_blocking=True)
# label = label.to(device=device, non_blocking=True)  

# optimizer.zero_grad()

# with autocast():
#     model_out = model(images, radar, mask_ratio=0)
#     model_out['labels'] = label

In [15]:
# with autocast():    
#     if isinstance(loss, list):
#         losses = {}
#         for l in loss:
#             losses.update(l(**model_out, output_dict=True))
#     else:
#         losses = loss(**model_out, output_dict=True)

#     total_loss = sum(losses.values())
#     losses["loss"] = total_loss

# total_loss.backward()

### LP

In [23]:
device = torch.device(training_args.device)
autocast = get_autocast(training_args.precision)

optical_encoder = model.module.optical_encoder.eval()
radar_encoder = model.module.radar_encoder.eval()

In [36]:
dict_data = {}
for split in ['train', 'test']:
    split_dict = dict(radar_feature=[], optical_feature=[], label=[])
    dataloader = data[split].dataloader
    for i, batch in enumerate(tqdm(dataloader)):
        images, radar, label = batch['image'], batch['radar'], batch['label']
        images = images.to(device=device, non_blocking=True)
        radar = radar.to(device=device, non_blocking=True)

        with torch.no_grad(), autocast():
            optical_features = optical_encoder(images, return_dict=True)
            radar_features = radar_encoder(radar, return_dict=True)
            split_dict['radar_feature'].append(radar_features['cls_token'].detach().cpu().numpy())
            split_dict['optical_feature'].append(optical_features['cls_token'].detach().cpu().numpy())
            split_dict['label'].append(label.numpy())
    split_dict['radar_feature'] = np.concatenate(split_dict['radar_feature'], axis=0)
    split_dict['optical_feature'] = np.concatenate(split_dict['optical_feature'], axis=0)
    split_dict['label'] = np.concatenate(split_dict['label'], axis=0)
    dict_data[split] = split_dict
pickle.dump(dict_data, open(f'./results/{exp_name}.pkl', 'wb'))

 64%|██████▍   | 990/1537 [17:07<04:35,  1.98it/s]

: 

In [35]:
split_dict['radar_feature'].shape

(256, 384)